
# Botanist
An academical projet to recognise and classify writings from botanists

Github repo: [github.com/satche/botanist](https://github.com/satche/botanist/)

## Get the data

First, we'll unzip raw data of different botanists notebooks. There is a lot of images, so run it and go grab a coffee. We'll connect to your Google Drive so we can save some results and output. Make sure to change the directory according to your folder structure.

*Note: the training data won't be stored in your drive as it's heavy*

In [ ]:
import os
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
# GDRIVE_PATH = "/content/drive/My Drive/Colab/Botanist"

In [ ]:
# !mkdir "/data/"
# !unzip herbier.zip -d "/data/"

In [ ]:
# data_public = os.listdir('./herbier/data_public/')
# data_neuchatel = os.listdir('./herbier/data_neuchatel/')
# print(data_public)
# print(data_neuchatel)

In [ ]:
# import packages
import numpy as np
import os
import numpy as np
from PIL import Image


In [ ]:


WORD_DATA_PATH = "./herbier/data_public/words/"
METADATA_PATH = "./herbier/data_public/ascii/words.txt"

def load_words_data(data_path, metadata_path):
    data = []

    with open(metadata_path, 'r') as file:
        for line in file:
            if not line.startswith("#"):
                # a01-000u-00-00 ok 154 408 768 27 51 AT A
                components = line.strip().split(' ')
                # a01-000u-00-00
                word_id = components[0]
                # ok
                segmentation_result = components[1]
                # 154
                gray_level = int(components[2])
                # 408 768 27 51
                bounding_box = tuple(map(int, components[3:7]))
                # AT
                grammatical_tag = components[7]
                # A
                transcription = ' '.join(components[8:]) if len(components) > 8 else ''

                parts = word_id.split('-')
                subfolder = parts[0] + '-' + parts[1]  # a01-000u
                image_subfolder = parts[0]  # a01
                image_filename = f"{word_id}.png"  # a01-000u-00-00.png
                image_path = os.path.join(data_path, image_subfolder, subfolder, image_filename)

                if os.path.exists(image_path):
                    image = Image.open(image_path)
                    img_array = np.array(image)
                    print(f"Loaded image for word ID: {word_id} at {image_path};")
                    data.append({
                        'word_id': word_id,
                        'segmentation_result': segmentation_result,
                        'gray_level': gray_level,
                        'bounding_box': bounding_box,
                        'grammatical_tag': grammatical_tag,
                        'transcription': transcription,
                        'image_array': img_array  # numpy arraz ? 
                    })
                    image.close()
                    
                       
                else:
                    print(f"Image not found for word ID: {word_id} at {image_path}")

    return data

words_data = load_words_data(WORD_DATA_PATH, METADATA_PATH)

print(f"Loaded {len(words_data)} words.")
for entry in words_data[:5]:
    print(f"  {entry['word_id']}: {entry['transcription']}; image shape: {entry['image_array'].shape}")

